In [ ]:
from google.colab import drive

# drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [1]:
import os
os.getcwd()

'/content'

In [ ]:
!pip install -q pandas

In [ ]:
!pip install -q bitsandbytes transformers accelerate datasets peft

In [ ]:
!pip install -q torch==2.6.0

In [ ]:
!pip install -q python-multipart fastapi uvicorn nest-asyncio pyngrok python-dotenv ftfy lxml_html_clean newspaper3k huggingface_hub

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Using CPU.")

GPU is available!
GPU Name: NVIDIA L4


In [ ]:
!nvidia-smi


Fri Jun 20 12:21:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   46C    P8             16W /   72W |       3MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv("drive/MyDrive/google_colab/MT/secrets.env")
# authtoken = os.environ.get("HUGGINGFACE_HUB_TOKEN")
# print(authtoken)

True

In [ ]:
# # to download a model from hugging face

# from huggingface_hub import login
# login(os.getenv("HUGGINGFACE_TOKEN"))

# from huggingface_hub import snapshot_download

# local_dir = "drive/MyDrive/google_colab/MT/models/granite-3.3-8b-instruct"

# snapshot_download(
#     repo_id="ibm-granite/granite-3.3-8b-instruct",
#     local_dir=local_dir,
#     local_dir_use_symlinks=False,
#     token=True
# )

In [ ]:
import re
import ftfy
import unicodedata
import pandas as pd
from newspaper import Article

def is_valid_email(email: str) -> bool:
    """Validate email format using regex."""
    email_pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    return re.match(email_pattern, email) is not None

def clean_text(text):
    if not text or not text.strip():  # Check for None, empty, or whitespace-only input
        return text  # Return as is (or return an empty string if preferred)

    # Fix text encoding issues
    text = ftfy.fix_text(text)

    # Normalize unicode characters (e.g., fix apostrophes, ellipses, etc.)
    text = unicodedata.normalize("NFC", text)

    return text

def extract_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.title, article.text


def data_pre_processing(folder_name, filename):
    file_location = os.path.join(folder_name, filename)
    data_v1 = pd.read_csv(file_location)
    data_v2 = data_v1[['url', 'title', 'content', 'lang']]

    exception_counter = 0
    for index, entity in data_v2.iterrows():
        try:
            title, content = extract_article(entity['url'])
            data_v2.at[index, "extracted_title"] = clean_text(title)
            data_v2.at[index, "extracted_content"] = clean_text(content)
            data_v2.at[index, "title"] = clean_text(data_v2.at[index, "title"])
            data_v2.at[index, "content"] = clean_text(data_v2.at[index, "content"])
            data_v2.at[index, "content_extracted"] = True
            data_v2.at[index, "error"] = ""
        except Exception as e:
            exception_counter += 1
            data_v2.at[index, "content_extracted"] = False
            data_v2.at[index, "error"] = f"{e}"
            # print(f'----------------------------------------------------------------')

    print(f"Number of exceptions: {exception_counter}/{len(data_v2)}")
    print(f"% of exceptions: {exception_counter*100/len(data_v2)} %")

    data_v2.to_json(f"{folder_name}/processed_data.json", orient="records", lines=True, force_ascii=False)

def print_line():
    print("----------------------------------------------------")

def calcuate_execution_time(start_time):
    end_time = time.time()
    processing_time = time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))
    print(f"Processing time: {processing_time}")

In [ ]:
questions_12 = [
      "What is the overall sentiment regarding Switzerland? (Very negative, Negative, Neutral, Positive, Very positive)",
      "Which aspect of the natural dimension is discussed? (Nature Dimension not addressed, Landscape/scenery, Geography, Weather/climate, Preserved nature, Nature activities, Other aspect of Nature dimension)",
      "What is the sentiment toward the natural dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
      "Which aspect of the functional dimension is discussed? (Functional Dimension not addressed, Education system, Science/innovation, Products, Economy, Infrastructure, Politics, Living/working conditions, Security, Other aspect of Functional dimension)",
      "What is the sentiment toward the functional dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
      "Which aspect of the normative dimension is discussed? (Normative Dimension not addressed, Environmental protection, Freedom/human rights, Civil rights, International engagement, Ethical issues/scandals, Conflict avoidance, Tolerance/openness, Other aspect of Normative dimension)",
      "What is the sentiment toward the normative dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
      "Which aspect of the cultural dimension is discussed? (Cultural Dimension not addressed, Sports, Food, Cultural offer, Personalities, Traditions, History, Cultural diversity, Other aspect of Culture dimension)",
      "What is the sentiment toward the cultural dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
      "Does the article contain disinformation? (No disinformation type, False connection, False context, Misleading content, Fabricated content, Manipulated content, Other disinformation type)",
      "What disinformation technique is used, if any? (No disinformation technique, Ad hominem attack, Emotional language, False dichotomies, Incoherence, Scapegoating, Other disinformation technique)",
      "What is the article about in short?"
    ]

columns_12 = [
        'overall_sentiment',
        'nature_dimension',
        'nature_sentiment',
        'functional_dimension',
        'functional_sentiment',
        'normative_dimension',
        'normative_sentiment',
        'cultural_dimension',
        'cultural_sentiment',
        'disinformation_type',
        'disinformation_technique',
        'summary'
    ]


questions_6 = [
        "What is the overall sentiment regarding Switzerland? (Very negative, Negative, Neutral, Positive, Very positive)",
        "What is the sentiment toward the natural dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
        "What is the sentiment toward the functional dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
        "What is the sentiment toward the normative dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
        "What is the sentiment toward the cultural dimension? (Very negative, Negative, Neutral, Positive, Very positive, No Sentiment)",
        "Does the article contain disinformation? (No disinformation type, False connection, False context, Misleading content, Fabricated content, Manipulated content, Other disinformation type)",
    ]

columns_6 = [
        'overall_sentiment',
        'nature_sentiment',
        'functional_sentiment',
        'normative_sentiment',
        'cultural_sentiment',
        'disinformation_type',
    ]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
def load_model(model_path):
  tokenizer = AutoTokenizer.from_pretrained(model_path)
  tokenizer.pad_token = tokenizer.eos_token

  model = AutoModelForCausalLM.from_pretrained(
      model_path,
      device_map="auto",
      torch_dtype="auto",
      load_in_8bit=True
  )

  return model, tokenizer



In [ ]:
import re
def process_by_llama_3b(model, tokenizer, input_file_path, questions, columns, output_file_path):

    data = pd.read_json(input_file_path, orient="records", lines=True)

    print(f"Data len: {len(data)}")

    data['article_source'] = ''
    for col in columns:
        data[col] = ''

    for index, row in data.iterrows():
        if not row['content_extracted'] or row['extracted_content'] == "" or len(row['content']) > len(row['extracted_content']):
            article = row['content']
            data.at[index, 'article_source'] = 'content'
        else:
            article = row['extracted_content']
            data.at[index, 'article_source'] = 'extracted_content'


        prompt = f"""
        You are given an article about Switzerland. Read it carefully.

        ARTICLE:
        {article}

        TASK:
        Answer the following multiple-choice questions based only on the information in the article.

        Instructions:
        - Choose **only one** of the options provided for each question.
        - Write only the selected option.
        - **Do not** explain your answer.

        QUESTIONS:
        """
        for i, q in enumerate(questions, 1):
            prompt += f"{i}. {q}\n"

        # Tokenize and send to model
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(model.device)
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=512,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                top_k=50
            )

        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        response_text = decoded.replace(prompt, "").strip()
        # print("--------------------------------------------------")
        # print("--------------------------------------------------")
        # print(f"Index: {index}")

        # print(f"Response: {response_text}")
        # print("--------------------------------------------------")
        # print("--------------------------------------------------")

        answer_lines = re.findall(r"^\s*(\d+)\.\s*(.*)$", response_text, flags=re.MULTILINE)
        answers = {int(num): ans.strip() for num, ans in answer_lines}

        # Fill in DataFrame columns
        for i, col in enumerate(columns, 1):
            data.at[index, col] = answers.get(i, "")



    # Save updated DataFrame to CSV
    data.to_csv(output_file_path, index=False, encoding='utf-8')
    print("Processing complete. Saved to CSV.")


In [ ]:
import re
import torch
import pandas as pd

def process_by_granite(model, tokenizer, input_file_path, questions, columns, output_file_path):
    # Load input data
    data = pd.read_json(input_file_path, orient="records", lines=True)
    print(f"Data length: {len(data)}")

    # Prepare new columns
    data['article_source'] = ''
    for col in columns:
        data[col] = ''

    for index, row in data.iterrows():
        # Select which article content to use
        if not row['content_extracted'] or row['extracted_content'] == "" or len(row['content']) > len(row['extracted_content']):
            article = row['content']
            data.at[index, 'article_source'] = 'content'
        else:
            article = row['extracted_content']
            data.at[index, 'article_source'] = 'extracted_content'

        # Build prompt for Granite 3.3-8B-Instruct
        prompt = f"""
        You are given an article about Switzerland. Read it carefully.

        ARTICLE:
        {article}

        TASK:
        Answer the following multiple-choice questions based only on the information in the article.

        Instructions:
        - Choose **only one** of the options provided for each question.
        - Write only the selected option.
        - **Do not** explain your answer.

        QUESTIONS:
        """
        for i, q in enumerate(questions, 1):
            prompt += f"{i}. {q}\n"

        # Tokenize and prepare inputs
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=4096).to(model.device)

        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=512,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                top_k=50
            )

        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        response_text = decoded[len(prompt):].strip()

        # print(f"Index: {index}")
        # print("--------------------------------------------------")

        # Extract answers
        answer_lines = re.findall(r"^\s*(\d+)\.\s*(.*)$", response_text, flags=re.MULTILINE)
        answers = {int(num): ans.strip() for num, ans in answer_lines}

        # Fill answers into dataframe
        for i, col in enumerate(columns, 1):
            data.at[index, col] = answers.get(i, "")

    # Save output
    data.to_csv(output_file_path, index=False, encoding='utf-8')


In [ ]:
import re
def process_by_mistral_7b(model, tokenizer, input_file_path, questions, columns, output_file_path):

    data = pd.read_json(input_file_path, orient="records", lines=True)

    print(f"Data len: {len(data)}")

    data['article_source'] = ''
    for col in columns:
        data[col] = ''

    for index, row in data.iterrows():
        if not row['content_extracted'] or row['extracted_content'] == "" or len(row['content']) > len(row['extracted_content']):
            article = row['content']
            data.at[index, 'article_source'] = 'content'
        else:
            article = row['extracted_content']
            data.at[index, 'article_source'] = 'extracted_content'

        prompt = f"""<s>[INST]
        You are given an article about Switzerland. Read it carefully.

        ARTICLE:
        {article}

        TASK:
        Answer the following multiple-choice questions based only on the information in the article.

        Instructions:
        - Choose **only one** of the options provided for each question.
        - Write only the selected option.
        - **Do not** explain your answer.

        QUESTIONS:
        """
        for i, q in enumerate(questions, 1):
            prompt += f"{i}. {q}\n"

        prompt += "[/INST]"


        # Tokenize and send to model
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(model.device)
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=512,
                pad_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                top_k=50
            )

        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        response_text = decoded.replace(prompt, "").strip()
        # print("--------------------------------------------------")
        # print("--------------------------------------------------")
        # print(f"Index: {index}")

        # print(f"Response: {response_text}")
        # print("--------------------------------------------------")
        # print("--------------------------------------------------")

        answer_lines = re.findall(r"^\s*(\d+)\.\s*(.*)$", response_text, flags=re.MULTILINE)
        answers = {int(num): ans.strip() for num, ans in answer_lines}

        # Fill in DataFrame columns
        for i, col in enumerate(columns, 1):
            data.at[index, col] = answers.get(i, "")



    # Save updated DataFrame to CSV
    data.to_csv(output_file_path, index=False, encoding='utf-8')
    print("Processing complete. Saved to CSV.")


In [ ]:
from datetime import datetime
data_count = 133
model, tokenizer = load_model("drive/MyDrive/google_colab/MT/models/Mistral-7B-Instruct-0.1")

print("Mistral-7B-Instruct-0.1")
for i in range(5):
    print(f"This is loop number {i + 1}")
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    print(timestamp)
    process_by_mistral_7b(model=model,
                  tokenizer=tokenizer,
                  input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
                  questions=questions_6,
                  columns=columns_6,
                  output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_mistral_7b_{data_count}_{timestamp}.csv")
    print(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    print("--------------------------------------------------")


In [ ]:
from datetime import datetime
data_count = 6
model, tokenizer = load_model("drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407-pre")

print("Mistral-Nemo-Instruct-2407")
for i in range(1):
    print(f"This is loop number {i + 1}")
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    print(timestamp)
    process_by_mistral_7b(model=model,
                  tokenizer=tokenizer,
                  input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
                  questions=questions_6,
                  columns=columns_6,
                  output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_mistral_nemo_{data_count}_{timestamp}.csv")
    print(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    print("--------------------------------------------------")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

ValueError: could not determine the shape of object type 'torch.storage.UntypedStorage'

In [ ]:
from datetime import datetime
data_count = 133
model, tokenizer = load_model("drive/MyDrive/google_colab/MT/models/Llama-3.2-3B-Instruct")

print("Llama-3.2-3B-Instruct")
print(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
for i in range(5):
    print(f"This is loop number {i + 1}")
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    print(timestamp)
    process_by_llama_3b(
      model=model,
      tokenizer=tokenizer,
      input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
      questions=questions_6,
      columns=columns_6,
      output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_llama_3_2_3b_{data_count}_{timestamp}.csv"
    )
    print(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    print("--------------------------------------------------")

In [ ]:
from datetime import datetime
data_count = 133
model, tokenizer = load_model("drive/MyDrive/google_colab/MT/models/granite-3.3-8b-instruct")

print("granite-3.3-8b-instruct")
for i in range(3):
    print(f"This is loop number {i + 1}")
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    print(timestamp)
    process_by_granite(
      model=model,
      tokenizer=tokenizer,
      input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
      questions=questions_6,
      columns=columns_6,
      output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_granite_3_3_8b_{data_count}_{timestamp}.csv"
    )
    print(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    print("--------------------------------------------------")

In [ ]:
# model_mistral_nemo, tokenizer_mistral_nemo = load_model_2("drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-FP8-2407")

In [ ]:
# model_llama_3_2_3b, tokenizer_llama_3_2_3b = load_model("drive/MyDrive/google_colab/MT/models/Llama-3.2-3B-Instruct")
# model_mistral_7b, tokenizer_mistral_7b = load_model("drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407")
model_granite_3_3_8b, tokenizer_granite_3_3_8b = load_model("drive/MyDrive/google_colab/MT/models/granite-3.3-8b-instruct")
# model_mistral_nemo, tokenizer_mistral_nemo = load_model("drive/MyDrive/google_colab/MT/models/Mistral-Nemo-Instruct-2407")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from datetime import datetime
data_count = 133
# model_granite_3_3_8b, tokenizer_granite_3_3_8b = load_model("drive/MyDrive/google_colab/MT/models/granite-3.3-8b-instruct")


for i in range(1):
    print(f"This is loop number {i + 1}")
    now = datetime.now()
    timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

    process_by_granite(
      model=model_granite_3_3_8b,
      tokenizer=tokenizer_granite_3_3_8b,
      input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
      questions=questions_6,
      columns=columns_6,
      output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_granite_3_3_8b_{data_count}_{timestamp}.csv"
    )

    # process_by_llama_3b(model=model_llama_3_2_3b,
    #                 tokenizer=tokenizer_llama_3_2_3b,
    #                 input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
    #                 questions=questions_6,
    #                 columns=columns_6,
    #                 output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_llama_3_2_3b_{data_count}_{timestamp}.csv")


    # process_by_mistral_7b(model=model_mistral_7b,
    #               tokenizer=tokenizer_mistral_7b,
    #               input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
    #               questions=questions_6,
    #               columns=columns_6,
    #               output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_mistral_nemo_{data_count}_{timestamp}.csv")








In [ ]:
# from datetime import datetime
# data_count = 133
# # model_llama_3_2_3b, tokenizer_llama_3_2_3b = load_model("drive/MyDrive/google_colab/MT/models/Llama-3.2-3B-Instruct")


# for i in range(1):
#     print(f"This is loop number {i + 1}")
#     now = datetime.now()
#     timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
#     process_by_llama_3b(model=model_llama_3_2_3b,
#                     tokenizer=tokenizer_llama_3_2_3b,
#                     input_file_path=f"drive/MyDrive/google_colab/MT/uploads/processed_data_{data_count}.json",
#                     questions=questions_6,
#                     columns=columns_6,
#                     output_file_path = f"drive/MyDrive/google_colab/MT/uploads/final_file_llama_3_2_3b_{data_count}_{timestamp}.csv")

This is loop number 1
Data len: 133


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Processing complete. Saved to CSV.


In [ ]:
# from transformers import pipeline

# # Initialize the text generation pipeline
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# # Example prompt
# prompt = "Please talk about Switzerland as a place to live."

# # Generate text based on the prompt
# output = pipe(prompt, max_new_tokens=150, temperature=0.7)

# # Print the generated text
# print(output[0]["generated_text"])

In [ ]:
import os
import shutil
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from dotenv import load_dotenv
from fastapi import FastAPI, UploadFile, File, Form
import time

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    # allow_origins=["http://localhost:4200"],  # Allow only your frontend origin
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],  # Allows all HTTP methods (GET, POST, etc.)
    allow_headers=["*"],  # Allows all headers
)

@app.get("/")
def health():
    start_time = time.time()
    print_line()
    calcuate_execution_time(start_time)
    return {"health get": "I'm fine habibi."}

@app.post("/")
def health():
    start_time = time.time()
    print_line()
    calcuate_execution_time(start_time)
    return {"health post": "I'm fine habibi."}

@app.post("/process")
async def process(
    file: UploadFile = File(...),
    model: str = Form(...)
    # email: str = Form(...)
):
    start_time = time.time()
    print_line()
    UPLOAD_FOLDER = "drive/MyDrive/google_colab/MT/uploads"

    # Check if the uploaded file is a CSV
    if not file.filename.endswith('.csv'):
        return JSONResponse(content={"Error": "Only CSV files are allowed!"}, status_code=400)

    if not os.path.exists(UPLOAD_FOLDER):
        os.makedirs(UPLOAD_FOLDER)

    # Define the path where the file will be saved
    file_location = os.path.join(UPLOAD_FOLDER, file.filename)

    # Save the uploaded file to the specified location
    with open(file_location, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    try:
        data_pre_processing(folder_name=UPLOAD_FOLDER, filename=file.filename)
    except Exception as e:
        calcuate_execution_time(start_time)
        return JSONResponse(content={"Error while pre-processing: ": str(e)}, status_code=500)


    calcuate_execution_time(start_time)
    return {
        "filename": file.filename,
        "message": "The file has been uploaded.",
        "model": model
    }

    # try:
    #     print(f"Model: {model}")
    #     match model:
    #         case "GPT-4o":
    #             print("case 1")
    #             process_by_gpt_4o(folder_name=UPLOAD_FOLDER, filename=file.filename)
    #         case "Llama-3.2-3B-Instruct":
    #             print("case 2")
    #             process_by_llama_mini()
    #         case _:
    #             print("case 0")
    #             return JSONResponse(content={f"Unvalid model: {model}."}, status_code=500)
    # except Exception as e:
    #     return JSONResponse(content={f"Error while processing using {model}: ": str(e)}, status_code=500)

    # try:
    #     return {
    #         "filename": file.filename,
    #         "message": "This is a test message.",
    #         "model": model
    #     }
    # except Exception as e:
    #     return JSONResponse(content={"error while returning the response: ": str(e)}, status_code=500)

In [ ]:
    # folder_path = "drive/MyDrive/google_colab/MT"
    # items = os.listdir(folder_path)

    # # Separate into files and directories
    # files = [f for f in items if os.path.isfile(os.path.join(folder_path, f))]
    # folders = [f for f in items if os.path.isdir(os.path.join(folder_path, f))]

    # print("Files:", files)
    # print("Folders:", folders)

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Allow nested event loops
nest_asyncio.apply()

# Expose the app via ngrok
public_url = ngrok.connect(8000)
print(f"Your app is live at: {public_url}")

# Run the server
uvicorn.run(app, host="127.0.0.1", port=8000)


In [ ]:
import time

# Store the start time
start_time = time.time()

# Simulate some processing (replace with your actual code)
time.sleep(5)  # Simulating 5 seconds of processing

# Calculate the processing time and format it as hh:mm:ss
processing_time = time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))

# Print the processing time
print(f"Processing time: {processing_time}")